In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from scipy.io import FortranFile
from cobaya.model import get_model
import yaml
import os
from getdist import plots, MCSamples
from cobaya.conventions import Const, packages_path_input
from scipy.optimize import minimize
from scipy.optimize import differential_evolution

In [ ]:
path_GLM_data = os.path.realpath(os.path.join(os.getcwd(), '../GLM_data'))

In [ ]:

getdist_settings = {
    'smooth_scale_1D': 0.3,
    'smooth_scale_2D': 0.4,
    'boundary_correction_order': 1,
    'mult_bias_correction_order': 1,
    }


# BAO

In [ ]:
yaml_path = os.path.realpath(os.path.join(os.getcwd(), '../chains/01p1_LCDM_BAO_DESIDR2.input.yaml'))

# define the path to the YAML parameters file:
yaml_params = yaml_path #os.path.join(chains_dir, name + '.input.yaml')
# load the parameters from the YAML file:
with open(yaml_params, 'r') as f:
    params_dict = yaml.safe_load(f)
# remove the 'path' key from the 'camb' dictionary if it exists:
if 'path' in params_dict.get('theory', {}).get('camb', {}):
    del params_dict['theory']['camb']['path']
if 'recombination_model' in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['recombination_model']
if 'max_zrei'in params_dict.get('theory', {}).get('camb', {}).get('extra_args', {}):
    del params_dict['theory']['camb']['extra_args']['max_zrei']
    print('Removed "max_zrei" from the "camb" extra_args in the YAML parameters file.')
# build the model:
model = get_model(params_dict)

In [ ]:
chains_dir = os.path.join('../chains')

# import chain
from cobaya import load_samples

name = '01p1_LCDM_BAO_DESIDR2'
chain = load_samples(os.path.join(chains_dir, name), skip=0.3, to_getdist=True)
chain.label = name
chain.root = os.path.join(chains_dir, name)
chain.updateSettings(getdist_settings)
chain.updateBaseStatistics()
chain.getConvergeTests()
print(chain.label+': R =', round(chain.GelmanRubin, 4))


BestFit = chain.getBestFit()
BestFitDict = BestFit.getParamDict()

params = model.parameterization.sampled_params()
for p in params.keys():
    params[p] = BestFitDict[p]  # Set the best fit values for all parameters

## Maximum Likelihood estimator

In [ ]:
BAO_pars = model.parameterization.sampled_params().keys()

def loglike_BAO(param_vec):
    par_dict = {}
    for i,key in enumerate(BAO_pars):
        par_dict[key] = param_vec[i]
    
    return model.loglike(par_dict, return_derived=False)


In [ ]:
differential_evolution?

In [ ]:
initial_guess = list(params.values())
bounds = [(-5, 5)] * len(initial_guess)
# result = minimize(loglike_BAO, initial_guess)

result = differential_evolution(loglike_BAO, bounds, polish=True, disp=True)

In [ ]:
cov_data_BAO = np.load(path_GLM_data + "cov_data_BAO.npy")

In [ ]:
Q_ML = - 2*result.fun - np.shape(cov_data_BAO)[0] * np.log(2*np.pi) - np.log(np.linalg.det(cov_data_BAO))
Q_ML

In [ ]:
result.x

In [ ]:
list(params.values())

## Maximum posterior estimator

In [ ]:
Q_MAP = -2 * loglike_BAO(list(params.values())) - np.shape(cov_data_BAO)[0] * np.log(2*np.pi) - np.log(np.linalg.det(cov_data_BAO))
Q_MAP